In [ ]:

class Controller(nn.Module):
    def __init__(self,input_dim,dim,output_dim,size):
        super(Controller,self).__init__()
        
        self.input_dim = input_dim # input size
        self.dim = dim # control dim
        self.output_dim = output_dim #output size
        self.size = size 
        
        self.controller = nn.LSTMCell(input_dim,dim) # controller
        
        self.out = nn.Linear(size,output_dim) # controller output
        
        torch.nn.init.xavier_uniform(self.out.weight) # init weight
        
        # hidden and cell states
        
        self.c_state = torch.zeros([1,dim])
        self.h_state = torch.zeros([1,dim])
        
        # learn init values 
        self.h_bias_fc = nn.Linear(1, dim)                                           
        self.c_bias_fc = nn.Linear(1, dim)
    
    def forward(self,x,prev_reads):
        
        
        x = torch.cat([x,*prev_reads],dim=1)
                
        self.h_state, self.c_state = self.controller(x, (self.h_state, self.c_state))
        
        return self.h_state, self.c_state
        
    def output(self,reads):
        
        out_state = torch.cat([self.h_state,*reads],dim=1)
        
        output = torch.sigmoid(self.out(out_state))
        
        return output  
        

In [ ]:
class Head(nn.Module):                                                                    
                                                                                          
    def __init__(self, mode, ctrl_dim, memory_unit_size):                                 
        super(Head, self).__init__()                                                      
        # Valid modes are 'r' and 'w' for reading and writing respectively                
        self.mode = mode                                                                  
        # Size of each memory vector (key size)                                           
        self.m = memory_unit_size                                                         
                                                                                          
        self.max_shift = 1                                                                
                                                                                          
        # Linear Layers for converting controller output to addressing parameters         
        self.key_fc = nn.Linear(ctrl_dim, self.m) 
        
        self.key_strength_fc = nn.Linear(ctrl_dim, 1) 
        """
        self.interpolation_gate_fc = nn.Linear(ctrl_dim, 1)                               
        self.shift_weighting_fc = nn.Linear(ctrl_dim, 3)                                  
        self.sharpen_factor_fc = nn.Linear(ctrl_dim, 1)                                   
        self.erase_weight_fc = nn.Linear(ctrl_dim, self.m)                                
        self.add_data_fc = nn.Linear(ctrl_dim, self.m) """    
    
    
    def forward(self, ctrl_state, prev_weights, memory):                                  
                                                                                                                                                               
        key = torch.tanh(self.key_fc(ctrl_state))                                         
        beta = F.softplus(self.key_strength_fc(ctrl_state))                                                                
                                                                                                                                                   
        # Content addressing                                                        
        content_weights = memory.content_addressing(key, beta)  
        
        if self.mode == 'r': 
            print("checking mode")
            read_vec = memory.read(weights)                                               
        # Write                                                                           
        elif self.mode == 'w':                                                            
            memory.write(weights, erase, add)                                             
            read_vec = None  
        else:
            raise ValueError("must me 'r' or 'w'")
    
  
    

In [ ]:
  class Memory(nn.Module):                                                                  
                                                                                          
        def __init__(self, memory, size):                                   
            super(Memory, self).__init__()                                                    


            self.n = memory                                                           

            self.m = size                                                         

            self.memory = torch.zeros([1, self.n, self.m])                                    

                                                                      
            
            
  class NTM(nn.Module):
    
        def __init__(self,                                                                    
                 input_dim,                                                               
                 output_dim,                                                              
                 control_dim,                                                                
                 m_unit,                                                            
                 m_size,                                                        
                 num_heads):                                                              
            super(NTM, self).__init__()  

            # Controller

            self.control_dim = control_dim

            #self.controller = Controller(input_dim + m_unit,control_dim,output_dim,control_dim+m_size)
            self.controller = Controller(input_dim ,control_dim,output_dim,control_dim)

            # Memory (Cols X Rows)

            self.memory = Memory(m_unit,m_size)
            self.m_size = m_size
            self.m_unit = m_unit

            # Single Head 

            self.read_head = Head('r',control_dim,m_size)
            self.write_head = Head('w',control_dim,m_size)

            # init prev heads 

            self.prev_head_weights = []                                                       
            self.prev_reads = []      

            # Layers to initialize the weights and read vectors                               
            self.head_weights_fc = nn.Linear(1, self.m_unit)                            
            self.reads_fc = nn.Linear(1, self.m_size) 
        
        def forward(self, x):                                                                 

            # Get controller states                                                           
            ctrl_hidden, ctrl_cell = self.controller(x, self.prev_reads)                      

            # Read, and Write                                                                 
            reads = None                                                                      
            head_weights = None
            
            
            print(len(self.heads))
            print(self.prev_head_weights)
                                               
            # Read                                                                                                                               
            weights, read_vec = read_head(ctrl_cell, prev_head_weights, self.memory)       
            reads = read_vec                                                   
                
            # Write                                                                                                                             
            weights, _ = write_head(ctrl_cell, prev_head_weights, self.memory)              

            head_weights = weights
                
            # Compute output                                                                  
            output = self.controller.output(reads)                                            

            self.prev_head_weights = head_weights                                             
            self.prev_reads = reads                                                           

            return output 
                
